In [15]:
import sqlite3
import pandas as pd
import numpy as np

# 1. Connect to an in-memory database
# (Use 'indian_ratings.db' for a file-based database that persists)
conn = sqlite3.connect('indian_ratings.db')
cursor = conn.cursor()

# 2. Drop the table if it already exists to start fresh
cursor.execute('DROP TABLE IF EXISTS indian_ratings')

# 3. Create a new table with the PRIMARY KEY constraint
cursor.execute('''
CREATE TABLE indian_ratings (
    user_id TEXT,
    movie_id TEXT,
    rating INTEGER,
    PRIMARY KEY (user_id, movie_id)
);
''')
conn.commit()

# 4. Generate sample data
users = ['Rahul', 'Priya', 'Amit', 'Pooja', 'Sanjay', 'Deepika', 'Akash', 'Shruti', 'Vikram', 'Anjali']
movies = ['Sholay', '3 Idiots', 'Dangal', 'Baahubali 2', 'Lagaan', 'Gully Boy', 'Bajirao Mastani', 'RRR', 'K.G.F: Chapter 2']
np.random.seed(42)

data_to_insert = []
# Ensure a sufficient number of unique ratings
while len(data_to_insert) < 50:
    user = np.random.choice(users)
    movie = np.random.choice(movies)
    rating = np.random.randint(1, 6) # Ratings from 1 to 5

    # Check for uniqueness before adding to the list
    if (user, movie) not in [(d[0], d[1]) for d in data_to_insert]:
        data_to_insert.append((user, movie, rating))

# 5. Insert the generated data into the table
cursor.executemany('INSERT INTO indian_ratings VALUES (?, ?, ?)', data_to_insert)
conn.commit()

print(f"Database created and populated with {len(data_to_insert)} ratings.")

# Close the connection for now
conn.close()

Database created and populated with 50 ratings.


In [16]:
import sqlite3
import pandas as pd

# Re-open the connection to the database file created in Part 1
conn = sqlite3.connect('indian_ratings.db')

# 1. Query all ratings into a pandas DataFrame
query = "SELECT user_id, movie_id, rating FROM indian_ratings"
long_format_ratings = pd.read_sql_query(query, conn)

print("Ratings loaded from database:")
print(long_format_ratings.head())
print(f"Total entries: {len(long_format_ratings)}")

# 2. Save the DataFrame to a CSV file
csv_file_path = 'indian_ratings.csv'
long_format_ratings.to_csv(csv_file_path, index=False)

print(f"\nData successfully saved to '{csv_file_path}'")

# Close the database connection
conn.close()

Ratings loaded from database:
   user_id         movie_id  rating
0    Akash      Baahubali 2       5
1   Shruti           Lagaan       2
2     Amit  Bajirao Mastani       3
3   Shruti              RRR       3
4  Deepika           Lagaan       2
Total entries: 50

Data successfully saved to 'indian_ratings.csv'


In [17]:
import pandas as pd
import numpy as np

# Load the data from the CSV file created in Part 2
csv_file_path = 'indian_ratings.csv'
long_format_ratings = pd.read_csv(csv_file_path)

# Reshape the data from long format to wide format (User-Item Matrix)
ratings = long_format_ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

print("User-Item Matrix (wide format):")
print(ratings.head())

# Compute User Similarities using Pearson Correlation
similarity_matrix = ratings.T.corr(method='pearson')
print("\nUser-User Pearson Correlation Similarity Matrix:")
print(similarity_matrix.head())

# k-NN: Find Similar Users
def get_k_nearest_neighbors(sim_matrix, user, k=2):
    sim_scores = sim_matrix[user].drop(user)
    return sim_scores.sort_values(ascending=False).head(k)

# Generate Recommendations
def recommend_items(ratings, similarity_matrix, user, k=2):
    neighbors = get_k_nearest_neighbors(similarity_matrix, user, k)
    weighted_scores = pd.Series(dtype=float)

    for neighbor, sim_score in neighbors.items():
        neighbor_ratings = ratings.loc[neighbor]
        weighted_scores = weighted_scores.add(neighbor_ratings * sim_score, fill_value=0)

    user_rated = ratings.loc[user]
    recommendations = weighted_scores[user_rated == 0].sort_values(ascending=False)
    return recommendations

# Example: Get recommendations for a random user from the database
random_user = np.random.choice(ratings.index)
print(f"\nRecommendations for {random_user}:")
recommendations = recommend_items(ratings, similarity_matrix, random_user, k=2)
print(recommendations)

User-Item Matrix (wide format):
movie_id  3 Idiots  Baahubali 2  Bajirao Mastani  Dangal  Gully Boy  \
user_id                                                               
Akash          4.0          5.0              5.0     0.0        0.0   
Amit           0.0          0.0              3.0     0.0        4.0   
Anjali         0.0          4.0              4.0     4.0        5.0   
Deepika        0.0          3.0              5.0     0.0        0.0   
Pooja          4.0          5.0              4.0     0.0        5.0   

movie_id  K.G.F: Chapter 2  Lagaan  RRR  Sholay  
user_id                                          
Akash                  0.0     1.0  3.0     0.0  
Amit                   0.0     3.0  3.0     3.0  
Anjali                 0.0     0.0  0.0     0.0  
Deepika                0.0     2.0  1.0     0.0  
Pooja                  0.0     0.0  4.0     1.0  

User-User Pearson Correlation Similarity Matrix:
user_id     Akash      Amit    Anjali   Deepika     Pooja     Priya   